In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

### Dataset

In [7]:
input_channels = 3
num_classes = 10

device = 'cuda'
lr = 0.01
epochs = 30
batch_size = 16

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(28)
])

def norm_image(data_sample):
    img_tensor = data_sample[0]
    label = data_sample[1]

    img_means = img_tensor.mean(axis=[1,2])
    img_sds = img_tensor.std(axis=[1,2])

    mean_sub = img_tensor - img_means.unsqueeze(1).unsqueeze(2)
    img_norm = mean_sub.true_divide(img_sds.unsqueeze(1).unsqueeze(2))

    return (img_norm, label)

### Training and Testing

In [8]:
all_train = list(datasets.CIFAR10(root = 'data/', transform=img_transform, train = True, download=True))

random.shuffle(all_train)

train_data = all_train[:40000]
train_transformed = list(map(norm_image, train_data))
train_loader = DataLoader(dataset=train_transformed, batch_size=batch_size, shuffle=True)

val_data = all_train[40000:]
val_transformed = list(map(norm_image, val_data))
val_loader = DataLoader(dataset=val_transformed, batch_size=batch_size, shuffle=True)

test_data = datasets.CIFAR10(root='data/', transform=img_transform, train=False, download=True)
test_transformed = list(map(norm_image, val_data))
test_loader = DataLoader(dataset=val_transformed, batch_size=batch_size)

100%|██████████| 170498071/170498071 [00:05<00:00, 31212109.49it/s]


Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


## Implementation of Reduced GoogLeNet

In [9]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU() # ReLU
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

class inception_block(nn.Module):
    def __init__(self, in_channels, out_ch1, out_ch3):
        super(inception_block, self).__init__()
        self.ch1 = conv_block(in_channels=in_channels, out_channels=out_ch1, kernel_size=(3,3), stride=(1,1), padding='same')
        self.ch3 = conv_block(in_channels=in_channels, out_channels=out_ch3, kernel_size=(3,3), stride=(1,1), padding='same')

    def forward(self,x):
        return torch.cat([self.ch1(x),self.ch3(x)],1)

class downsample_block(nn.Module):
    def __init__(self, in_channels, conv_out):
        super(downsample_block, self).__init__()
        self.convblock = conv_block(in_channels, conv_out, kernel_size=(3,3), stride=(2,2))
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))

    def forward(self,x):
        return torch.cat([self.convblock(x),self.maxpool(x)],1)

class mini_GoogLeNet(nn.Module):
    def __init__(self,in_channels=3, num_classes=10, dropout_prob=0):
        super(mini_GoogLeNet, self).__init__()
        self.conv1 = conv_block(in_channels=3, out_channels=96, kernel_size=(3,3), stride=(1,1))
        self.inception1 = inception_block(96,32,32)
        self.inception2 = inception_block(64,32,48)
        self.downsample1 = downsample_block(80,80)
        self.inception3 = inception_block(160,112,48)
        self.inception4 = inception_block(160,96,64)
        self.inception5 = inception_block(160,80,80)
        self.inception6 = inception_block(160,48,96)
        self.downsample2 = downsample_block(144,96)
        self.inception7 = inception_block(240,176,160)
        self.inception8 = inception_block(336,176,160)
        self.avgpool = nn.AvgPool2d(kernel_size=(7,7), padding=(1,1))
        self.dropout = nn.Dropout(p = dropout_prob)
        self.fc = nn.Linear(336,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.downsample1(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.inception5(x)
        x = self.inception6(x)
        x = self.downsample2(x)
        x = self.inception7(x)
        x = self.inception8(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

## Training Model

In [10]:
model = mini_GoogLeNet(in_channels=3, num_classes=10, dropout_prob=0).to(device=device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.LinearLR(optimizer)

In [11]:
def train(epoch):
    model.train()
    curr_loss_train = 0
    correct_train = 0
    total_train = 0

    for ind, (data_train, true_labels_train) in enumerate(train_loader):
        data_train = data_train.to(device=device)
        true_labels_train = true_labels_train.to(device=device)
        out_train = model(data_train)
        loss_train = loss_function(out_train, true_labels_train)
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        curr_loss_train += loss_train.item()
        ix, predicted_train = out_train.max(1)
        correct_train += predicted_train.eq(true_labels_train).sum().item()
        total_train += true_labels_train.size(0)

    train_loss = curr_loss_train/len(train_loader)
    acc_train_val = (correct_train/total_train)*100
    train_acc.append(acc_train_val)
    train_all_loss.append(train_loss)

def test(epoch):
    model.eval()
    curr_loss_test = 0
    correct_test = 0
    total_test = 0
    num_class = 10
    confusion_matrix = torch.zeros(num_class, num_class)
    with torch.no_grad():
        for data_test, true_labels_test in test_loader:

            data_test = data_test.to(device=device)
            true_labels_test = true_labels_test.to(device=device)

            out_test = model(data_test)
            loss_test = loss_function(out_test, true_labels_test)

            curr_loss_test += loss_test.item()
            ix, predicted_test = out_test.max(1)
            correct_test += predicted_test.eq(true_labels_test).sum().item()
            total_test += true_labels_test.size(0)


    test_loss = curr_loss_test/len(test_loader)
    acc_test_val = (correct_test/total_test)*100
    test_acc.append(acc_test_val)
    test_all_loss.append(test_loss)
    con_mats.append(confusion_matrix)

### Performance Metrics

In [23]:
train_acc = []
train_all_loss = []
test_acc = []
test_all_loss = []
con_mats = []
times = []
train_times = []

In [13]:
train_start = time.time()
for epoch in range(epochs):
    ep_start = time.time()
    print(f"Epoch {epoch}")
    train(epoch)
    test(epoch)
    scheduler.step()

    epoch_time = time.time() - ep_start
    print(f"Epoch time: {epoch_time:0.2f} seconds")
    times.append(epoch_time)

    train_time = time.time() - train_start
    train_times.append(train_time)

print()
print(f"Total training time: {train_time:0.2f} seconds")

Epoch 0
Epoch time: 35.48 seconds
Epoch 1
Epoch time: 33.97 seconds
Epoch 2
Epoch time: 34.72 seconds
Epoch 3
Epoch time: 34.60 seconds
Epoch 4
Epoch time: 34.49 seconds
Epoch 5
Epoch time: 34.82 seconds
Epoch 6
Epoch time: 35.38 seconds
Epoch 7
Epoch time: 34.52 seconds
Epoch 8
Epoch time: 34.50 seconds
Epoch 9
Epoch time: 34.72 seconds
Epoch 10
Epoch time: 34.84 seconds
Epoch 11
Epoch time: 34.06 seconds
Epoch 12
Epoch time: 34.47 seconds
Epoch 13
Epoch time: 34.44 seconds
Epoch 14
Epoch time: 34.81 seconds
Epoch 15
Epoch time: 34.11 seconds
Epoch 16
Epoch time: 34.65 seconds
Epoch 17
Epoch time: 34.55 seconds
Epoch 18
Epoch time: 34.64 seconds
Epoch 19
Epoch time: 34.16 seconds
Epoch 20
Epoch time: 34.33 seconds
Epoch 21
Epoch time: 34.47 seconds
Epoch 22
Epoch time: 34.44 seconds
Epoch 23
Epoch time: 34.19 seconds
Epoch 24
Epoch time: 34.35 seconds
Epoch 25
Epoch time: 34.70 seconds
Epoch 26
Epoch time: 34.33 seconds
Epoch 27
Epoch time: 34.65 seconds
Epoch 28
Epoch time: 34.51 sec

## Exporting results

In [18]:
df_res = pd.DataFrame()
df_res['TrainAccuracy'] = train_acc
df_res['TrainLoss'] = train_all_loss
df_res['TestAccuracy'] = test_acc
df_res['TestLoss'] = test_all_loss
df_res['EpochTime'] = times
df_res['Totaltime'] = train_times

df_res.to_csv('results.csv', index=False)

In [19]:
with open('confusion_matrices.pickle','wb') as handle:
    pickle.dump(con_mats, handle)

## Results

In [20]:
pd.set_option('display.max_rows', None)
df_res

,TrainAccuracy,TrainLoss,TestAccuracy,TestLoss,EpochTime,Totaltime
0,43.4900,1.628851,56.29,1.270878,35.479408,35.479617
1,61.4450,1.129382,66.66,0.973625,33.970527,69.450256
2,69.4250,0.894084,72.79,0.809873,34.719829,104.171005
3,74.9600,0.745310,72.30,0.821674,34.603584,138.774688
4,78.0850,0.649898,76.70,0.682987,34.491376,173.266178
5,80.6225,0.567858,80.54,0.587196,34.817836,208.084156
6,83.8800,0.478076,78.94,0.642532,35.381085,243.465339
7,86.6725,0.398423,81.22,0.590557,34.523232,277.988676
8,88.8100,0.332564,80.03,0.623583,34.498762,312.487535
9,90.7025,0.277501,79.09,0.706202,34.717371,347.206336


### Best Model Accuracy

In [22]:
max_accuracy = df_res['TestAccuracy'].max()
print(f"Peak accuracy: {max_accuracy}%")

Peak accuracy: 84.15%
